# CI/CD for a KFP pipeline

**Learning Objectives:**
1. Learn how to create a custom Cloud Build builder to pilot CAIP Pipelines
1. Learn how to write a Cloud Build config file to build and push all the artifacts for a KFP
1. Learn how to setup a Cloud Build Github trigger to rebuild the KFP

In this lab you will walk through authoring of a **Cloud Build** CI/CD workflow that automatically builds and deploys a KFP pipeline. You will also integrate your workflow with **GitHub** by setting up a trigger that starts the  workflow when a new tag is applied to the **GitHub** repo hosting the pipeline's code.




In [ ]:
# Create required GKE cluster
gcloud container clusters create cluster-1 \
--zone us-central1-a \
--cluster-version 1.21.5-gke.1302 \
--machine-type n1-standard-2 \
--scopes=https://www.googleapis.com/auth/cloud-platform

## Configuring environment settings

Update  the `ENDPOINT` constat  with the settings reflecting your lab environment. 

Then endpoint to the AI Platform Pipelines instance can be found on the [AI Platform Pipelines](https://console.cloud.google.com/ai-platform/pipelines/clusters) page in the Google Cloud Console.

1. Open the *SETTINGS* for your instance
2. Use the value of the `host` variable in the *Connect to this Kubeflow Pipelines instance from a Python client via Kubeflow Pipelines SKD* section of the *SETTINGS* window.

In [1]:
ENDPOINT = 'https://77bbd65bbd0b74b5-dot-us-central1.pipelines.googleusercontent.com'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

## Creating the KFP CLI builder



### Exercise

In the cell below, write a docker file that
* Uses `gcr.io/deeplearning-platform-release/base-cpu` as base image
* Install the python package `kfp` with version `0.2.5`
* Starts `/bin/bash` as entrypoint

In [3]:
%%writefile kfp-cli/Dockerfile
FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install kfp==0.2.5
ENTRYPOINT ["/bin/bash"]

Overwriting kfp-cli/Dockerfile


### Build the image and push it to your project's **Container Registry**.

In [4]:
IMAGE_NAME='kfp-cli'
TAG='latest'
IMAGE_URI='gcr.io/{}/{}:{}'.format(PROJECT_ID, IMAGE_NAME, TAG)

### Exercise

In the cell below, use `gcloud builds` to build the `kfp-cli` Docker image and push it to the project gcr.io registry.

In [5]:
!gcloud builds submit --timeout 15m --tag {IMAGE_URI} kfp-cli

Creating temporary tarball archive of 1 file(s) totalling 103 bytes before compression.
Uploading tarball of [kfp-cli] to [gs://qwiklabs-gcp-00-8962bb682264_cloudbuild/source/1642924130.219413-9cf6987f087d434d903f7b571c4c3a23.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-00-8962bb682264/locations/global/builds/9f32b423-b466-4483-8c4d-118a47c18587].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/9f32b423-b466-4483-8c4d-118a47c18587?project=334587390474].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "9f32b423-b466-4483-8c4d-118a47c18587"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-00-8962bb682264_cloudbuild/source/1642924130.219413-9cf6987f087d434d903f7b571c4c3a23.tgz#1642924130950072
Copying gs://qwiklabs-gcp-00-8962bb682264_cloudbuild/source/1642924130.219413-9cf6987f087d434d903f7b571c4c3a23.tgz#1642924130950072...
/ [1 files][  225.0 B/  225.0 B]                       

## Understanding the **Cloud Build** workflow.

### Exercise

In the cell below, you'll complete the `cloudbuild.yaml` file describing the CI/CD workflow and prescribing how environment specific settings are abstracted using **Cloud Build** variables.

The CI/CD workflow automates the steps you walked through manually during `lab-02-kfp-pipeline`:
1. Builds the trainer image
1. Builds the base image for custom components
1. Compiles the pipeline
1. Uploads the pipeline to the KFP environment
1. Pushes the trainer and base images to your project's **Container Registry**

Although the KFP backend supports pipeline versioning, this feature has not been yet enable through the **KFP** CLI. As a temporary workaround, in the **Cloud Build** configuration a value of the `TAG_NAME` variable is appended to the name of the pipeline. 

The **Cloud Build** workflow configuration uses both standard and custom [Cloud Build builders](https://cloud.google.com/cloud-build/docs/cloud-builders). The custom builder encapsulates **KFP CLI**. 

In [ ]:
%%writefile trainer_image/train.py
""""Covertype Classifier trainer script."""

import pickle
import subprocess
import sys

import fire
import hypertune
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

def train_evaluate(job_dir, training_dataset_path, validation_dataset_path,
    alpha, max_iter, hptune):
    """Trains the Covertype Classifier model."""

    df_train = pd.read_csv(training_dataset_path)
    df_validation = pd.read_csv(validation_dataset_path)

    if not hptune:
        df_train = pd.concat([df_train, df_validation])

    numeric_features = [
      'Elevation', 'Aspect', 'Slope', 'Horizontal_Distance_To_Hydrology',
      'Vertical_Distance_To_Hydrology', 'Horizontal_Distance_To_Roadways',
      'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
      'Horizontal_Distance_To_Fire_Points'
    ]
    
    categorical_features = ['Wilderness_Area', 'Soil_Type']

    preprocessor = ColumnTransformer(transformers=[(
        'num', StandardScaler(),
        numeric_features), ('cat', OneHotEncoder(), categorical_features)])

    pipeline = Pipeline([('preprocessor', preprocessor),
        ('classifier', SGDClassifier(loss='log'))])

    num_features_type_map = {feature: 'float64' for feature in numeric_features}
    df_train = df_train.astype(num_features_type_map)
    df_validation = df_validation.astype(num_features_type_map)

    print('Starting training: alpha={}, max_iter={}'.format(alpha, max_iter))
    X_train = df_train.drop('Cover_Type', axis=1)
    y_train = df_train['Cover_Type']

    pipeline.set_params(classifier__alpha=alpha, classifier__max_iter=max_iter)
    pipeline.fit(X_train, y_train)

    if hptune:
        X_validation = df_validation.drop('Cover_Type', axis=1)
        y_validation = df_validation['Cover_Type']
        accuracy = pipeline.score(X_validation, y_validation)
        print('Model accuracy: {}'.format(accuracy))
        # Log it with hypertune
        hpt = hypertune.HyperTune()
        hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag='accuracy', metric_value=accuracy)

    # Save the model
    if not hptune:
        model_filename = 'model.pkl'
        with open(model_filename, 'wb') as model_file:
            pickle.dump(pipeline, model_file)
        gcs_model_path = '{}/{}'.format(job_dir, model_filename)
        subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path],
            stderr=sys.stdout)
        print('Saved model in: {}'.format(gcs_model_path))

if __name__ == '__main__':
    fire.Fire(train_evaluate)

In [ ]:
%%writefile trainer_image/Dockerfile
FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire cloudml-hypertune scikit-learn==0.20.4 pandas==0.24.2
WORKDIR /app
COPY train.py .

ENTRYPOINT ["python", "train.py"]

In [ ]:
%%writefile base_image/Dockerfile
FROM gcr.io/deeplearning-platform-release/base-cpu
RUN pip install -U fire scikit-learn==0.20.4 pandas==0.24.2 kfp==0.2.5

In [ ]:
%%writefile pipeline/helper_components.py
"""Helper components."""

from typing import NamedTuple

def retrieve_best_run(project_id: str, job_id: str) -> NamedTuple('Outputs', 
    [('metric_value', float), ('alpha', float),('max_iter', int)]):
    """Retrieves the parameters of the best Hypertune run."""

    from googleapiclient import discovery
    from googleapiclient import errors

    ml = discovery.build('ml', 'v1')

    job_name = 'projects/{}/jobs/{}'.format(project_id, job_id)
    request = ml.projects().jobs().get(name=job_name)

    try:
        response = request.execute()
    except errors.HttpError as err:
        print(err)
    except:
        print('Unexpected error')

    print(response)

    best_trial = response['trainingOutput']['trials'][0]

    metric_value = best_trial['finalMetric']['objectiveValue']
    alpha = float(best_trial['hyperparameters']['alpha'])
    max_iter = int(best_trial['hyperparameters']['max_iter'])

    return (metric_value, alpha, max_iter)

def evaluate_model(dataset_path: str, model_path: str, metric_name: str) -> NamedTuple(
    'Outputs', [('metric_name', str), ('metric_value', float),
        ('mlpipeline_metrics', 'Metrics')]):
    """Evaluates a trained sklearn model."""
    #import joblib
    import pickle
    import json
    import pandas as pd
    import subprocess
    import sys

    from sklearn.metrics import accuracy_score, recall_score

    df_test = pd.read_csv(dataset_path)

    X_test = df_test.drop('Cover_Type', axis=1)
    y_test = df_test['Cover_Type']

    # Copy the model from GCS
    model_filename = 'model.pkl'
    gcs_model_filepath = '{}/{}'.format(model_path, model_filename)
    print(gcs_model_filepath)
    subprocess.check_call(['gsutil', 'cp', gcs_model_filepath, model_filename],
        stderr=sys.stdout)

    with open(model_filename, 'rb') as model_file:
        model = pickle.load(model_file)

    y_hat = model.predict(X_test)

    if metric_name == 'accuracy':
        metric_value = accuracy_score(y_test, y_hat)
    elif metric_name == 'recall':
        metric_value = recall_score(y_test, y_hat)
    else:
        metric_name = 'N/A'
        metric_value = 0

    # Export the metric
    metrics = {
      'metrics': [{
          'name': metric_name,
          'numberValue': float(metric_value)
      }]
    }

    return (metric_name, metric_value, json.dumps(metrics))

In [ ]:
%%writefile pipeline/covertype_training_pipeline.py
"""KFP pipeline orchestrating BigQuery and Cloud AI Platform services."""

import os
from helper_components import evaluate_model
from helper_components import retrieve_best_run
from jinja2 import Template
import kfp
from kfp.components import func_to_container_op
from kfp.dsl.types import Dict
from kfp.dsl.types import GCPProjectID
from kfp.dsl.types import GCPRegion
from kfp.dsl.types import GCSPath
from kfp.dsl.types import String
from kfp.gcp import use_gcp_secret

# Defaults and environment settings
BASE_IMAGE = os.getenv('BASE_IMAGE')
TRAINER_IMAGE = os.getenv('TRAINER_IMAGE')
RUNTIME_VERSION = os.getenv('RUNTIME_VERSION')
PYTHON_VERSION = os.getenv('PYTHON_VERSION')
COMPONENT_URL_SEARCH_PREFIX = os.getenv('COMPONENT_URL_SEARCH_PREFIX')
USE_KFP_SA = os.getenv('USE_KFP_SA')

TRAINING_FILE_PATH = 'datasets/training/data.csv'
VALIDATION_FILE_PATH = 'datasets/validation/data.csv'
TESTING_FILE_PATH = 'datasets/testing/data.csv'

# Parameter defaults
SPLITS_DATASET_ID = 'splits'
HYPERTUNE_SETTINGS = """
{
    "hyperparameters":  {
        "goal": "MAXIMIZE",
        "maxTrials": 6,
        "maxParallelTrials": 3,
        "hyperparameterMetricTag": "accuracy",
        "enableTrialEarlyStopping": True,
        "params": [
            {
                "parameterName": "max_iter",
                "type": "DISCRETE",
                "discreteValues": [500, 1000]
            },
            {
                "parameterName": "alpha",
                "type": "DOUBLE",
                "minValue": 0.0001,
                "maxValue": 0.001,
                "scaleType": "UNIT_LINEAR_SCALE"
            }
        ]
    }
}
"""

# Helper functions
def generate_sampling_query(source_table_name, num_lots, lots):
    """Prepares the data sampling query."""

    sampling_query_template = """
        SELECT *
        FROM 
            `{{ source_table }}` AS cover
        WHERE 
        MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), {{ num_lots }}) IN ({{ lots }})
    """
    query = Template(sampling_query_template).render(
        source_table=source_table_name, num_lots=num_lots, lots=str(lots)[1:-1])

    return query

# Create component factories
component_store = kfp.components.ComponentStore(
    local_search_paths=None, url_search_prefixes=[COMPONENT_URL_SEARCH_PREFIX])

bigquery_query_op = component_store.load_component('bigquery/query')
mlengine_train_op = component_store.load_component('ml_engine/train')
mlengine_deploy_op = component_store.load_component('ml_engine/deploy')
retrieve_best_run_op = func_to_container_op(
    retrieve_best_run, base_image=BASE_IMAGE)
evaluate_model_op = func_to_container_op(evaluate_model, base_image=BASE_IMAGE)

@kfp.dsl.pipeline(
    name='Covertype Classifier Training',
    description='The pipeline training and deploying the Covertype classifierpipeline_yaml'
)
def covertype_train(project_id,
                    region,
                    source_table_name,
                    gcs_root,
                    dataset_id,
                    evaluation_metric_name,
                    evaluation_metric_threshold,
                    model_id,
                    version_id,
                    replace_existing_version,
                    hypertune_settings=HYPERTUNE_SETTINGS,
                    dataset_location='US'):
    """Orchestrates training and deployment of an sklearn model."""

    # Create the training split
    query = generate_sampling_query(
        source_table_name=source_table_name, num_lots=10, lots=[1, 2, 3, 4])

    training_file_path = '{}/{}'.format(gcs_root, TRAINING_FILE_PATH)

    create_training_split = bigquery_query_op(
        query=query,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id='',
        output_gcs_path=training_file_path,
        dataset_location=dataset_location)

    # Create the validation split
    query = generate_sampling_query(
        source_table_name=source_table_name, num_lots=10, lots=[8])

    validation_file_path = '{}/{}'.format(gcs_root, VALIDATION_FILE_PATH)

    create_validation_split = bigquery_query_op(
        query=query,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id='',
        output_gcs_path=validation_file_path,
        dataset_location=dataset_location)

    # Create the testing split
    query = generate_sampling_query(
        source_table_name=source_table_name, num_lots=10, lots=[9])

    testing_file_path = '{}/{}'.format(gcs_root, TESTING_FILE_PATH)

    create_testing_split = bigquery_query_op(
        query=query,
        project_id=project_id,
        dataset_id=dataset_id,
        table_id='',
        output_gcs_path=testing_file_path,
        dataset_location=dataset_location)

    # Tune hyperparameters
    tune_args = [
        '--training_dataset_path',
        create_training_split.outputs['output_gcs_path'],
        '--validation_dataset_path',
        create_validation_split.outputs['output_gcs_path'], '--hptune', 'True'
    ]

    job_dir = '{}/{}/{}'.format(gcs_root, 'jobdir/hypertune',
        kfp.dsl.RUN_ID_PLACEHOLDER)

    hypertune = mlengine_train_op(
        project_id=project_id,
        region=region,
        master_image_uri=TRAINER_IMAGE,
        job_dir=job_dir,
        args=tune_args,
        training_input=hypertune_settings)

    # Retrieve the best trial
    get_best_trial = retrieve_best_run_op(project_id, hypertune.outputs['job_id'])

    # Train the model on a combined training and validation datasets
    job_dir = '{}/{}/{}'.format(gcs_root, 'jobdir', kfp.dsl.RUN_ID_PLACEHOLDER)

    train_args = [
        '--training_dataset_path',
        create_training_split.outputs['output_gcs_path'],
        '--validation_dataset_path',
        create_validation_split.outputs['output_gcs_path'], '--alpha',
        get_best_trial.outputs['alpha'], '--max_iter',
        get_best_trial.outputs['max_iter'], '--hptune', 'False'
    ]

    train_model = mlengine_train_op(
        project_id=project_id,
        region=region,
        master_image_uri=TRAINER_IMAGE,
        job_dir=job_dir,
        args=train_args)

    # Evaluate the model on the testing split
    eval_model = evaluate_model_op(
        dataset_path=str(create_testing_split.outputs['output_gcs_path']),
        model_path=str(train_model.outputs['job_dir']),
        metric_name=evaluation_metric_name)

    # Deploy the model if the primary metric is better than threshold
    with kfp.dsl.Condition(
        eval_model.outputs['metric_value'] > evaluation_metric_threshold):
    deploy_model = mlengine_deploy_op(
        model_uri=train_model.outputs['job_dir'],
        project_id=project_id,
        model_id=model_id,
        version_id=version_id,
        runtime_version=RUNTIME_VERSION,
        python_version=PYTHON_VERSION,
        replace_existing_version=replace_existing_version)

    # Configure the pipeline to run using the service account defined
    # in the user-gcp-sa k8s secret
    if USE_KFP_SA == 'True':
        kfp.dsl.get_pipeline_conf().add_op_transformer(use_gcp_secret('user-gcp-sa'))

In [6]:
%%writefile cloudbuild.yaml

steps:
# Build the trainer image
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME', '.']
  dir: $_PIPELINE_FOLDER/trainer_image

# Build the base image for lightweight components
- name: 'gcr.io/cloud-builders/docker'
  args: ['build', '-t', 'gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME', '.']
  dir: $_PIPELINE_FOLDER/base_image

# Compile the pipeline
# Set the environment variables below for the $_PIPELINE_DSL script
# HINT: https://cloud.google.com/cloud-build/docs/configuring-builds/substitute-variable-values
- name: 'gcr.io/$PROJECT_ID/kfp-cli'
  args:
  - '-c'
  - |
    dsl-compile --py $_PIPELINE_DSL --output $_PIPELINE_PACKAGE
  env:
  - 'BASE_IMAGE=gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME'
  - 'TRAINER_IMAGE=gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME'
  - 'RUNTIME_VERSION=$_RUNTIME_VERSION'
  - 'PYTHON_VERSION=$_PYTHON_VERSION'
  - 'COMPONENT_URL_SEARCH_PREFIX=$_COMPONENT_URL_SEARCH_PREFIX'
  - 'USE_KFP_SA=$_USE_KFP_SA'
  dir: $_PIPELINE_FOLDER/pipeline

# Upload the pipeline
# Use the kfp-cli Cloud Builder and write the command to upload the ktf pipeline 
- name: 'gcr.io/$PROJECT_ID/kfp-cli'
  args:
  - '-c'
  - |
    kfp --endpoint $_ENDPOINT pipeline upload -p ${_PIPELINE_NAME}_$TAG_NAME $_PIPELINE_PACKAGE
  dir: $_PIPELINE_FOLDER/pipeline


# Push the images to Container Registry
# List the images to be pushed to the project Docker registry
images: ['gcr.io/$PROJECT_ID/$_TRAINER_IMAGE_NAME:$TAG_NAME', 
         'gcr.io/$PROJECT_ID/$_BASE_IMAGE_NAME:$TAG_NAME']

Overwriting cloudbuild.yaml


## Manually triggering CI/CD runs

You can manually trigger **Cloud Build** runs using the `gcloud builds submit` command.

In [7]:
SUBSTITUTIONS="""
_ENDPOINT={},\
_TRAINER_IMAGE_NAME=trainer_image,\
_BASE_IMAGE_NAME=base_image,\
TAG_NAME=test,\
_PIPELINE_FOLDER=.,\
_PIPELINE_DSL=covertype_training_pipeline.py,\
_PIPELINE_PACKAGE=covertype_training_pipeline.yaml,\
_PIPELINE_NAME=covertype_continuous_training,\
_RUNTIME_VERSION=1.15,\
_PYTHON_VERSION=3.7,\
_USE_KFP_SA=True,\
_COMPONENT_URL_SEARCH_PREFIX=https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/
""".format(ENDPOINT).strip()

In [8]:
!gcloud builds submit . --config cloudbuild.yaml --substitutions {SUBSTITUTIONS}

Creating temporary tarball archive of 9 file(s) totalling 79.3 KiB before compression.
Uploading tarball of [.] to [gs://qwiklabs-gcp-00-8962bb682264_cloudbuild/source/1642924693.051181-cdf8977a4d5a4c42bd2699392fca8b0e.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/qwiklabs-gcp-00-8962bb682264/locations/global/builds/576ded26-6ad3-461b-b999-1bc2f8fc0bc7].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/576ded26-6ad3-461b-b999-1bc2f8fc0bc7?project=334587390474].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "576ded26-6ad3-461b-b999-1bc2f8fc0bc7"

FETCHSOURCE
Fetching storage object: gs://qwiklabs-gcp-00-8962bb682264_cloudbuild/source/1642924693.051181-cdf8977a4d5a4c42bd2699392fca8b0e.tgz#1642924693264234
Copying gs://qwiklabs-gcp-00-8962bb682264_cloudbuild/source/1642924693.051181-cdf8977a4d5a4c42bd2699392fca8b0e.tgz#1642924693264234...
/ [1 files][ 19.1 KiB/ 19.1 KiB]                              

## Setting up GitHub integration

### Exercise

In this exercise you integrate your CI/CD workflow with **GitHub**, using [Cloud Build GitHub App](https://github.com/marketplace/google-cloud-build). 
You will set up a trigger that starts the CI/CD workflow when a new tag is applied to the **GitHub** repo managing the  pipeline source code. You will use a fork of this repo as your source GitHub repository.

#### Step 1: Create a fork of this repo
[Follow the GitHub documentation](https://help.github.com/en/github/getting-started-with-github/fork-a-repo) to fork this repo

#### Step 2: Create a **Cloud Build** trigger

Connect the fork you created in the previous step to your Google Cloud project and create a trigger following the steps in the [Creating GitHub app trigger](https://cloud.google.com/cloud-build/docs/create-github-app-triggers) article. Use the following values on the **Edit trigger** form:

|Field|Value|
|-----|-----|
|Name|[YOUR TRIGGER NAME]|
|Description|[YOUR TRIGGER DESCRIPTION]|
|Event| Tag|
|Source| [YOUR FORK]|
|Tag (regex)|.\*|
|Build Configuration|Cloud Build configuration file (yaml or json)|
|Cloud Build configuration file location|/ workshops/kfp-caip-sklearn/lab-03-kfp-cicd/cloudbuild.yaml|


Use the following values for the substitution variables:

|Variable|Value|
|--------|-----|
|_BASE_IMAGE_NAME|base_image|
|_COMPONENT_URL_SEARCH_PREFIX|https://raw.githubusercontent.com/kubeflow/pipelines/0.2.5/components/gcp/|
|_ENDPOINT|[Your inverting proxy host]|
|_PIPELINE_DSL|covertype_training_pipeline.py|
|_PIPELINE_FOLDER|workshops/kfp-caip-sklearn/lab-03-kfp-cicd|
|_PIPELINE_NAME|covertype_training_deployment|
|_PIPELINE_PACKAGE|covertype_training_pipeline.yaml|
|_PYTHON_VERSION|3.7|
|_RUNTIME_VERSION|1.15|
|_TRAINER_IMAGE_NAME|trainer_image|
|_USE_KFP_SA|False|

### Trigger the build

To start an automated build [create a new release of the repo in GitHub](https://help.github.com/en/github/administering-a-repository/creating-releases). Alternatively, you can start the build by applying a tag using `git`. 
```
git tag [TAG NAME]
git push origin --tags
```


<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>